In [1]:
import os

from torch.utils.data import DataLoader

from model import Dataset
from model import collate_fn, build_pretrained_model

# 1. Argument 설정

## 1.1 Dataset Path

In [2]:
dataset_path = 'dataset'

train_query_path = os.path.join(dataset_path, 'smart_src.txt')
train_gold_reply_path = os.path.join(dataset_path, 'smart_tgt.txt')
train_gen_reply_path = None

train_data_path_list = [train_query_path, train_gold_reply_path, train_gold_reply_path]

# 1.2 OpenNMT Path
OpenNMT Training시 사용 된 vocab.pt 파일과 Pretraining된 모델의 경로

In [3]:
onmt_path = 'sample_onmt'

onmt_vocab_path = os.path.join(onmt_path, 'sample.vocab.pt')
onmt_model_path = os.path.join(onmt_path, 'sample.model.pt')

# 1.3 Training Parameter

In [9]:
batch_size = 4

# 2. 객체 생성

# 2.1 Dataset instance 생성
Dataset 구축을 위해 OpenNMT vocab과 data file path list를 넘겨줌.

In [5]:
# Training dataset 구축
my_dataset = Dataset(
    vocab_path=onmt_vocab_path,
    data_path_list=train_data_path_list,
    max_length=35
)

[*] Loading onmt vocab dictionary...
Number of dictionary : 23786 

[*] Reading lines...
Avg length of data 0 : 6.81
Avg length of data 1 : 6.81
Avg length of data 2 : 6.81

Read 90729 sentence pairs
Trim data to 90701 sentence pairs 

[*] Success to preprocess data! 



# 2.2 DataLoader instance 생성
실제 학습 시 DataLoader 객체를 다음과 같이 iteration 시키면 알아서 Batch data를 iterate 해줌.

```python
for data1, data1_len, data2, data2_len in dataloader_instance:
    # padding & packing 등 필요한 작업
    # model 학습
```

DataLoader에서 던져주는 값들은 **각각의 데이터의 Batch와 Length들의 반복**으로 이루어져 있음. 예를 들어 위의 Example 상황 같은 경우, Batch size가 16이고 Query와 Gold reply, 2개의 데이터로 이루어져 있으므로 다음과 같이 던져주게 됨.

* Query sentence 4개 tuple
* Query sentence length 4개 tuple
* Gold reply sentence 4개 tuple
* Gold reply sentence length 4개 tuple

만약 Gen reply까지 Dataset에 포함을 시키면 총 6개의 instance를 iterate 해줌

In [10]:
my_data_loader = DataLoader(
    dataset=my_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn,
    num_workers=16
)

In [11]:
# Example
next(iter(my_data_loader))

[([6, 5, 55, 3],
  [54, 78, 3],
  [6, 5, 470, 898, 458, 471, 5, 3207, 12, 777, 8, 67, 3],
  [6, 5, 9, 11, 674, 0, 12170, 2]),
 (4, 3, 13, 8),
 ([54, 78, 3],
  [6, 5, 470, 898, 458, 471, 5, 3207, 12, 777, 8, 67, 3],
  [6, 5, 9, 11, 674, 0, 12170, 2],
  [6, 2]),
 (3, 13, 8, 2),
 ([54, 78, 3],
  [6, 5, 470, 898, 458, 471, 5, 3207, 12, 777, 8, 67, 3],
  [6, 5, 9, 11, 674, 0, 12170, 2],
  [6, 2]),
 (3, 13, 8, 2)]

# 2.3 OpenNMT 모델 로드
OpenNMT의 pretrained embedding과 rnn의 parameter를 땡겨옴.

Sample은 Embedding dimension 10, RNN size 15의 Tiny model임.

In [12]:
encoder = build_pretrained_model(onmt_model_path, my_dataset.vocab)

Loading pretrained model... 

RNNEncoder(
  (embeddings): Embeddings(
    (make_embedding): Sequential(
      (emb_luts): Elementwise(
        (0): Embedding(23786, 10, padding_idx=1)
      )
    )
  )
  (rnn): GRU(10, 15, bidirectional=True)
) 

